In [ ]:
from influxdb import InfluxDBClient
import time;
import random

json_body = []

for j in range(1, 20):
    for x in range(1, 100):
        time.sleep(0.1)
        ticks = round(time.time()*1000) # time in ms
        print(ticks)
        json_body += [
            {
                "measurement": "kek",
                "time": ticks*1000*1000,
                "fields": {
                    "value": random.random(),
                    "value2": random.random()
                }
            }
        ]
    
    client = InfluxDBClient('localhost', 8086, 'root', 'root', 'statsdemo')
    client.write_points(json_body)

In [75]:
import json
import sys
import matplotlib.pyplot as plt
%matplotlib inline  
file = open("last100s.txt")

def flatten(data, prefix=""):
    flat = dict()
    
    if isinstance(data,dict):
        if prefix:
            prefix += "."
        for k,v in data.items():
            flat.update(flatten(v, prefix + str(k)))
        return flat;
    
    if isinstance(data,list):
        if prefix:
            prefix += "."
        i = 0;
        for v in data:
            flat.update(flatten(v, prefix + str(i)))
            i += 1
        return flat
    
    return {prefix:data}



json_body = []
start_time = round(time.time()*1000) # time in ms

id = []
stack = []
bat = {"cellA":[],"cellB":[]}
for line in file:
    try:
        data = json.loads(line)
        if data["id"] == "sensor":
            try:
                
                flat = flatten(data)
                flat["id"] = 0.0 #Need to handle enumerated states!
                flt = dict()
                
                for key in flat:
                    flt[key] = float(flat[key])
                    #print(flat)
                
                json_body += [
                    {
                        "measurement": "flight1",
                        "time": (start_time + data[u"tick"])*1000*1000,
                        "fields": flt
                    }
                ]
                
                
                
            except KeyError:
                print('keyError')
                print(line)
        if data[u"id"] == u"status":
            tick_stat.append(data[u"tick"])
            #bat["cell"].append(data[u'bat'][u"cell"])
    except ValueError:
        print('line not JSON')
file.close()

client = InfluxDBClient('localhost', 8086, 'root', 'root', 'statsdemo')
client.write_points(json_body)


True